In [ ]:
#!/usr/bin/python

from mininet.net import Mininet
from mininet.node import Controller, RemoteController, OVSController
from mininet.node import CPULimitedHost, Host, Node
from mininet.node import OVSKernelSwitch, UserSwitch
from mininet.node import IVSSwitch
from mininet.cli import CLI
from mininet.log import setLogLevel, info
from mininet.link import TCLink, Intf
from subprocess import call

def myNetwork():

    net = Mininet( topo=None,
                   build=False,
                   ipBase='10.0.0.0/8')

    info( '*** Adding controller\n' )
    c0=net.addController(name='c0',
                      controller=Controller,
                      protocol='tcp',
                      port=6653)

    info( '*** Add switches\n')
    s1 = net.addSwitch('s1', cls=OVSKernelSwitch)

    info( '*** Add hosts\n')
    h2 = net.addHost('h2', cls=Host, ip='10.0.0.2', defaultRoute=None)
    h1 = net.addHost('h1', cls=Host, ip='10.0.0.1', defaultRoute=None)

    info( '*** Add links\n')
    net.addLink(s1, h1)
    net.addLink(s1, h2)

    info( '*** Starting network\n')
    net.build()
    info( '*** Starting controllers\n')
    for controller in net.controllers:
        controller.start()

    info( '*** Starting switches\n')
    net.get('s1').start([c0])

    info( '*** Post configure switches and hosts\n')
    hosts = net.hosts
    server = hosts[ 1 ]
    outfiles, capfiles, errfiles = {}, {}, {}

    for h in hosts:
        h.cmdPrint('IP address of', h) #, h.name.IP())
        outfiles[ h ] = './simpleNet/out/%s.out' % h.name # to store the output of ping command for client to server
        capfiles[ h ] = './simpleNet/cap/%s.txt' % h.name #cap file to store the output of tcpdump command
        errfiles[ h ] = './simpleNet/err/%s.err' % h.name
       
    newHosts = {hosts[ 0 ]} # set h1 as a ping sender, i.e., client
    h1 = {hosts[ 0 ]} # set h1 as a ping sender, i.e., client
    h2 = {hosts[ 1 ]} # set h1 as a ping sender, i.e., client

    serverHost = {hosts [ 1 ]} # set h2 as a ping receiver, i.e., server

    for h in serverHost:
        h.cmdPrint('tcpdump -n -i h1-eth0',
                 '>', capfiles[ h ],
                 '2>', errfiles[ h ],
                 '&' )

    


    for h in newHosts:
        h.cmdPrint('ping -c 15', server.IP(),
                 '>', outfiles[ h ],
                 '2>', errfiles[ h ]
                 )# '&' )



    CLI(net)
    net.stop()

if __name__ == '__main__':
    setLogLevel( 'info' )
    myNetwork()